In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import base64
import pytesseract
from io import BytesIO
import pytesseract
from PIL import Image
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
url="https://www.mahatenders.gov.in/nicgep/app?page=FrontEndAdvancedSearch&service=page"
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')
driver= webdriver.Chrome()
driver.get(r"https://www.mahatenders.gov.in/nicgep/app?page=FrontEndAdvancedSearch&service=page")

In [3]:
def clean_captch(extracted_text):
    demo=""
    for i in extracted_text:
        if i.isalpha():
            demo +=i
        elif i.isdigit():
            demo +=i
    return demo

In [4]:
def find_data_tender(tender_links):
    cap=['Organisation Chain', 'Tender Reference Number', 'Tender ID', 'Withdrawal Allowed', 'Tender Type', 'Form Of Contract', 'Tender Category', 'No. of Covers', 'General Technical Evaluation Allowed', 'ItemWise Technical Evaluation Allowed', 'Payment Mode', 'Is Multi Currency Allowed For BOQ', 'Is Multi Currency Allowed For Fee', 'Allow Two Stage Bidding', 'Tender Fee in ₹', 'Fee Payable To', 'Fee Payable At', 'Tender Fee Exemption Allowed', 'EMD Amount in ₹', 'EMD through BG/ST or EMD Exemption Allowed', 'EMD Fee Type', 'EMD Percentage', 'EMD Payable To', 'EMD Payable At', 'Title', 'Work Description', 'NDA/Pre Qualification', 'Independent External Monitor/Remarks', 'Tender Value in ₹', 'Product Category', 'Sub category', 'Contract Type', 'Bid Validity(Days)', 'Period Of Work(Days)', 'Location', 'Pincode', 'Pre Bid Meeting Place', 'Pre Bid Meeting Address', 'Pre Bid Meeting Date', 'Bid Opening Place', 'Should Allow NDA Tender', 'Allow Preferential Bidder', 'Published Date', 'Bid Opening Date', 'Document Download / Sale Start Date', 'Document Download / Sale End Date', 'Clarification Start Date', 'Clarification End Date', 'Bid Submission Start Date', 'Bid Submission End Date', 'Name', 'Address']
    fei=[]
    url = tender_links
    if not url.startswith('https://www.mahatenders.gov.in'):
        url = 'https://www.mahatenders.gov.in' + url  # Add a default scheme if missing
    driver.get(url)

    # Wait for the content to load
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "table")))

    # Extract the page source HTML
    page_source = driver.page_source


    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(page_source, "html.parser")

    # Find all tables on the page
    tables = soup.find_all("table" , class_ = "tablebg")
    # Process the tables as needed
    cout=0
    for table in tables:
        abc = table.prettify() 
        table_soup = BeautifulSoup(abc, "html.parser")  # Convert the table string to a BeautifulSoup object
        cout +=1
        table_special=[2,3,8]
        if cout in table_special:
            continue
        else :
            #cap_demo = table_soup.find_all("td", class_="td_caption")
            #for i in cap_demo:
            #    cap.append(i.text.strip())  # Add the text to cap list if found

            fei_demo = table_soup.find_all("td", class_="td_field")
            for j in fei_demo:
                fei.append(j.text.strip())  # Add the text to fei list if found
            
        

    return fei
    
    
    #driver.quit()
            
        

   
    
    
    

In [5]:
def  data_in_tenders(soup):
    rows=soup.find("table", id = "table")
    td_tags = rows.find_all('td')
    cap=['Organisation Chain', 'Tender Reference Number', 'Tender ID', 'Withdrawal Allowed', 'Tender Type', 'Form Of Contract', 'Tender Category', 'No. of Covers', 'General Technical Evaluation Allowed', 'ItemWise Technical Evaluation Allowed', 'Payment Mode', 'Is Multi Currency Allowed For BOQ', 'Is Multi Currency Allowed For Fee', 'Allow Two Stage Bidding', 'Tender Fee in ₹', 'Fee Payable To', 'Fee Payable At', 'Tender Fee Exemption Allowed', 'EMD Amount in ₹', 'EMD through BG/ST or EMD Exemption Allowed', 'EMD Fee Type', 'EMD Percentage', 'EMD Payable To', 'EMD Payable At', 'Title', 'Work Description', 'NDA/Pre Qualification', 'Independent External Monitor/Remarks', 'Tender Value in ₹', 'Product Category', 'Sub category', 'Contract Type', 'Bid Validity(Days)', 'Period Of Work(Days)', 'Location', 'Pincode', 'Pre Bid Meeting Place', 'Pre Bid Meeting Address', 'Pre Bid Meeting Date', 'Bid Opening Place', 'Should Allow NDA Tender', 'Allow Preferential Bidder', 'Published Date', 'Bid Opening Date', 'Document Download / Sale Start Date', 'Document Download / Sale End Date', 'Clarification Start Date', 'Clarification End Date', 'Bid Submission Start Date', 'Bid Submission End Date', 'Name', 'Address']
    data=[]

# Extract URLs from <a> tags within <td> tags
    urls = [td.a['href'] for td in td_tags if td.a]
    url_demo = []
    for tender_links in urls:
        url_demo.append(tender_links)
    data_ten=[]  
    for tender_link in url_demo:
        data_ten.append(find_data_tender(tender_link))
        
    df= pd.DataFrame(data=data_ten, columns=cap)
    print(df)
    df.to_csv('tender_scrapping.csv')
        
    

In [6]:
count = 0
for i in range(10):
    print("*"*10, i)

    input_element= driver.find_element(By.ID,"TenderType")
    input_element.send_keys("Open Tender")

    refresh = driver.find_element(By.XPATH, '//*[@id="captcha"]')
    refresh.click()

    time.sleep(10)

    base64_image_url = driver.find_element(By.XPATH, '//*[@id="captchaImage" and @name="captchaImage"]').get_attribute("src")
    base64_data = base64_image_url.split(",")[1]
    

    # Calculate the missing padding
    missing_padding = len(base64_data) % 4
    if missing_padding:
        base64_data += '=' * (4 - missing_padding)

    # Decode the base64 string
    image_data = base64.b64decode(base64_data)

    # Open the image using PIL
    image = Image.open(BytesIO(image_data))

    # Perform OCR to extract text from the image
    try :
        extracted_text = pytesseract.image_to_string(image)
        final_text = clean_captch(extracted_text)
        #print(f"Trying {i} times", final_text, extracted_text)
   
        # Write capcha
        input_element= driver.find_element(By.ID,"captchaText")
        input_element.send_keys(final_text)

        # Click search button
        element = driver.find_element(By.XPATH, '//*[@id="submit"]')
        element.click()

        incorrect_capcha = driver.find_element(By.XPATH, '//*[@id="TenderAdvancedSearch"]/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr[5]/td/table/tbody/tr/td')

        if incorrect_capcha:
            time.sleep(10)
            refresh = driver.find_element(By.XPATH, '//*[@id="captcha"]')
            refresh.click()
            print("Element exists.")
        else:
            print("Element does not exist.")



    except Exception as e:
        print(e)
       


    time.sleep(10)
    # Check if we have find correct capcha and break the loop
    soup = BeautifulSoup(driver.page_source)
    if "e-Published Date" in  soup.text:
        #create fun for extracting data
        data_in_tenders(soup)
        print("break statement executed")
        
        break
        
    else:
        print("e-Published Date does not exist.")

********** 0
Alert Text: Please enter Captcha.
Message: unexpected alert open: {Alert text : Please enter Captcha.}
  (Session info: chrome=121.0.6167.185)
Stacktrace:
	GetHandleVerifier [0x00007FF793055E42+3538674]
	(No symbol) [0x00007FF792C74C02]
	(No symbol) [0x00007FF792B25AEB]
	(No symbol) [0x00007FF792BAC55E]
	(No symbol) [0x00007FF792B8EE53]
	(No symbol) [0x00007FF792B5F514]
	(No symbol) [0x00007FF792B60631]
	GetHandleVerifier [0x00007FF793086CAD+3738973]
	GetHandleVerifier [0x00007FF7930DC506+4089270]
	GetHandleVerifier [0x00007FF7930D4823+4057299]
	GetHandleVerifier [0x00007FF792DA5C49+720121]
	(No symbol) [0x00007FF792C8126F]
	(No symbol) [0x00007FF792C7C304]
	(No symbol) [0x00007FF792C7C432]
	(No symbol) [0x00007FF792C6BD04]
	BaseThreadInitThunk [0x00007FFACA6D55A0+16]
	RtlUserThreadStart [0x00007FFACB26485B+43]

e-Published Date does not exist.
********** 1
Element exists.
e-Published Date does not exist.
********** 2


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=121.0.6167.185)
Stacktrace:
	GetHandleVerifier [0x00007FF793055E42+3538674]
	(No symbol) [0x00007FF792C74C02]
	(No symbol) [0x00007FF792B25AEB]
	(No symbol) [0x00007FF792B1273F]
	(No symbol) [0x00007FF792B122B0]
	(No symbol) [0x00007FF792B27BF1]
	(No symbol) [0x00007FF792BAB437]
	(No symbol) [0x00007FF792B8EE53]
	(No symbol) [0x00007FF792B5F514]
	(No symbol) [0x00007FF792B60631]
	GetHandleVerifier [0x00007FF793086CAD+3738973]
	GetHandleVerifier [0x00007FF7930DC506+4089270]
	GetHandleVerifier [0x00007FF7930D4823+4057299]
	GetHandleVerifier [0x00007FF792DA5C49+720121]
	(No symbol) [0x00007FF792C8126F]
	(No symbol) [0x00007FF792C7C304]
	(No symbol) [0x00007FF792C7C432]
	(No symbol) [0x00007FF792C6BD04]
	BaseThreadInitThunk [0x00007FFACA6D55A0+16]
	RtlUserThreadStart [0x00007FFACB26485B+43]
